In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from glassnode_utils import get_price_ohlc

import plotly.graph_objects as go
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

symbol = 'BTCUSD'
start = '2022-01-01'
end = '2023-06-11'
strategy_interval = '1h'
candles = 200
rolling_wave_length = 20
num_clusters = 4
adx_strength_threshold = 25

In [ ]:
data_all = get_price_ohlc(interval = strategy_interval)
#limit by date
data = data_all[start:end].copy()
#limit by number of candles
data = data[candles * -1:]

In [ ]:
def plot_all(levels, df, title):
  fig = go.Figure(data=go.Candlestick(x=df.index,
                open=df['OpenPrice'],
                high=df['HighPrice'],
                low=df['LowPrice'],
                close=df['ClosePrice']))

  last_price = df['ClosePrice'][-1]
  for level in levels:
    fig.add_annotation(x=df.index[-1], 
                       y=level[1], 
                       text='{:0.8f}'.format(level[1]),
                       yanchor='bottom',
                       font=dict(size=12, color="purple", family="Sans Serif"), 
                       align="left")

    line_color = 'red'
    if (last_price < level[1]):
      line_color = 'green'

    fig.add_shape(type="line", 
                  x0=df.index[level[0]], y0=level[1], 
                  x1=max(df.index), y1=level[1],
                  line_width=2, 
                  line_dash="dash", 
                  line_color=line_color)

  fig.update_layout(title=title, xaxis_rangeslider_visible=False, width=1400, height=800)
  fig.show()

In [ ]:
from support_resistance_utils import get_kmeans_levels, get_fractal_levels, get_window_shifting_levels, get_pivot_point_levels, get_agglomerative_clustering

plot_all(get_kmeans_levels(data), data, 'Support Resistance - Kmeans')
rolling_wave_length = 20
num_clusters = 4
plot_all(get_agglomerative_clustering(data, rolling_wave_length, num_clusters), data, 'Support Resistance - Agglomerative Clustering')
plot_all(get_fractal_levels(data), data, 'Support Resistance - Fractal')
plot_all(get_window_shifting_levels(data), data, 'Support Resistance - Shifting')
plot_all(get_pivot_point_levels(data), data, 'Support Resistance - Pivot Point')

In [ ]:
from machine_learning_utils import get_kmeans_clusters

"""
Based on:
https://medium.com/@judopro/using-machine-learning-to-programmatically-determine-stock-support-and-resistance-levels-9bb70777cf8e
https://realpython.com/k-means-clustering-python/
"""

lows = pd.DataFrame(data=data, index=data.index, columns=['LowPrice'])
highs = pd.DataFrame(data=data, index=data.index, columns=['HighPrice'])

low_clusters, lwcss = get_kmeans_clusters(lows)
low_centers = low_clusters.cluster_centers_
low_centers = np.sort(low_centers, axis=0)

k = low_centers.size
plt.plot(range(1, 11), lwcss)
plt.scatter(range(1, 11), lwcss)
plt.scatter(k, lwcss[k], marker='X', s = 300, c = 'r')
plt.title('The Elbow Method - Support')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

high_clusters, hwcss = get_kmeans_clusters(highs)
high_centers = high_clusters.cluster_centers_
high_centers = np.sort(high_centers, axis=0)

k = high_centers.size
plt.plot(range(1, 11), hwcss)
plt.scatter(range(1, 11), hwcss)
plt.scatter(k, hwcss[k], marker='X', s = 300, c = 'r')
plt.title('The Elbow Method - Resistance')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
from support_resistance_utils import get_all_pivot_point_levels

pivot_point, support_l1, support_l2, support_l3, resistance_l1, resistance_l2, resistance_l3 = get_all_pivot_point_levels(data)

num_candles = 30
fig = go.Figure(data=go.Candlestick(x=data.index[-num_candles:],
                open=data['OpenPrice'][-num_candles:],
                high=data['HighPrice'][-num_candles:],
                low=data['LowPrice'][-num_candles:],
                close=data['ClosePrice'][-num_candles:],
                showlegend=False))

fig.add_hline(y=pivot_point[-1], 
              line_dash='dash', 
              line_color='blue', 
              opacity=0.3,
              exclude_empty_subplots=True)

# Support level 1
fig.add_hline(y=support_l1[-1], 
              line_dash='dash', 
              line_color='red', 
              opacity=0.3,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=support_l1.index[-num_candles:], 
                         y=support_l1[-num_candles:],
                         line_color='red',
                         opacity=0.3,
                         name='support 1',
                         mode='lines'))

# Support Level 2
fig.add_hline(y=support_l2[-1], 
              line_dash='dash', 
              line_color='red', 
              opacity=0.5,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=support_l2.index[-num_candles:], 
                         y=support_l2[-num_candles:],
                         line_color='red',
                         opacity=0.5,
                         name='support 2',
                         mode='lines'))

# Support Level 3
fig.add_hline(y=support_l3[-1], 
              line_dash='dash', 
              line_color='red', 
              opacity=1,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=support_l3.index[-num_candles:], 
                         y=support_l3[-num_candles:],
                         line_color='red',
                         opacity=1,
                         name='support 3',
                         mode='lines'))

# Resistance level 1
fig.add_hline(y=resistance_l1[-1], 
              line_dash='dash', 
              line_color='green',
              opacity=0.3,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=resistance_l1.index[-num_candles:], 
                         y=resistance_l1[-num_candles:],
                         line_color='green',
                         opacity=0.3,
                         name='resistance 1',
                         mode='lines'))

# Resistance level 2
fig.add_hline(y=resistance_l2[-1], 
              line_dash='dash', 
              line_color='green',
              opacity=0.5,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=resistance_l2.index[-num_candles:], 
                         y=resistance_l2[-num_candles:],
                         line_color='green',
                         opacity=0.5,
                         name='resistance 2',
                         mode='lines'))

# Resistance level 3
fig.add_hline(y=resistance_l3[-1], 
              line_dash='dash', 
              line_color='green',
              opacity=1,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=resistance_l3.index[-num_candles:], 
                         y=resistance_l3[-num_candles:],
                         line_color='green',
                         opacity=1,
                         name='resistance 3',
                         mode='lines'))

fig.update_layout(title='Support - Resistance levels', xaxis_rangeslider_visible=False, width=1400, height=800)
fig.show()

In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from sklearn.cluster import AgglomerativeClustering

### Long Entry:

1. First, we need to determine the levels of support and resistance for a range of dates.
2. When find events when the price rises to a level of resistance and breaks through that level.
3. At the same time we will use the ADX indicator to determine the strength of the trend. We will check for a value of 25. This is to prevent that a weak price trend ends in a failed breakout.
4. We will also check the ADX positive (+DI) and negative (-DI) indicators for the price direction. In this case the +DI has to be above the -DI.


### Long Exit

In this strategy we are only interest in exploiting that first, explosive upward trend after the breakout. So as an exit strategy, I’m just going to use a trailing stop loss and stop loss combination.

* Trailing stop loss of 1%
* Stop loss of 5%.

https://medium.com/@chris_42047/implementing-a-support-resistance-breakout-strategy-python-tutorial-53b86bbd393d

In [ ]:
#  LONG ENTRY
#----------------------------------------------
#  calculate support/resistance levels
rolling_wave_length = 20
num_clusters = 4
if self.i < 200 + rolling_wave_length:
    return
#  Grab a subset of the data
support_resistance_lb = self.data.df.iloc[-200 - rolling_wave_length:]
#  Calculate support/resistance
support_resistance_levels = calculate_support_resistance(support_resistance_lb, rolling_wave_length, num_clusters)
#  Check resistance crossover
current_price = self.data.Close[-1]
for level in support_resistance_levels.to_list():
    if self.data.Close[-1] >= level and self.data.Close[-2] <= level:
        long_entry_signal += 1
        break
#  ADX trend strength
if self.adx[-1] >= self.adx_strength_threshold:
    long_entry_signal += 1

#  Check trend direction
adx_pos = self.adx_pos_neg[0]
adx_neg = self.adx_pos_neg[1]
if adx_pos[-1] > adx_neg[-1]:
    long_entry_signal += 1

#  LONG EXIT
#----------------------------------------------
#  Stop loss
if self.long_hold == 1 and current_price <= (self.last_purchase_price * (1 - (self.stop_loss_pc/100))):
    long_exit_signal += 1
#  Track max price after long entry
if self.long_hold == 1 and current_price > self.long_peak_price:
    self.long_peak_price = current_price
#  Trailing stop loss
trailing_stop_price = self.long_peak_price * (1 - (self.trailing_stop_pc/100))
if self.long_hold == 1 and current_price <= trailing_stop_price:
    long_exit_signal += 1

In [ ]:
def get_breakout_signal(
        price_data,
        adx,
        di_plus,
        di_minus,
        rolling_wave_length,
        num_clusters,
        adx_threshold=25.0,
        buy_first=True,
        remove_repeated_signals=True):
    
    signal = price_data.copy()
    signal = 0.0

    i = 0
    j = candles

    if (len(signal) < j):
        return signal

    while (j < len(signal)):
        levels = get_agglomerative_clustering(price_data[i:j], rolling_wave_length, num_clusters)

        current_price = price_data[j]
        previous_price = price_data[j - 1]

        for level in levels:
            #  LONG ENTRY
            if (current_price >= level[1]
                    and previous_price <= level[1]
                    and adx.iloc[j - 1] >= adx_threshold
                    and di_plus.iloc[j - 1] > di_minus.iloc[j - 1]):
                signal.iloc[j] = 1.0
                break

            #  SHORT ENTRY
            if (current_price <= level[1]
                    and previous_price >= level[1]
                    and adx.iloc[j - 1] >= adx_threshold
                    and di_plus.iloc[j - 1] < di_minus.iloc[j - 1]):
                signal.iloc[j] = -1.0
                break

        i += 1
        j += 1

    return signal

In [ ]:
df = data_all['2021-09-09':]
df = get_breakout_signal(df, rolling_wave_length, num_clusters)

print(df[df['signal'] != 0])

In [ ]:
#levels = get_fractal_levels(data)
#if (has_breakout(levels[-5:], data.iloc[-2], data.iloc[-1])):
#  print('Breakout...')

# get the coin list
# for coin get prices, apply methods